<a href="https://colab.research.google.com/github/ttjh1234/CSproject_SDA/blob/main/posedetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PoseDetection 사용

## 필요 라이브러리 임포트

In [ ]:
!pip install mediapipe==0.8.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 67.0 MB 93 kB/s 
     |████████████████████████████████| 14.9 MB 403 kB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xarray-einstats 0.2.2 requires numpy>=1.21, but you have numpy 1.19.3 which is incompatible.
tensorflow 2.8.2+zzzcolab20220527125636 requires numpy>=1.20, but you have numpy 1.19.3 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import zipfile
import os
import tensorflow as tf
from tensorflow import keras
import re
import mediapipe as mp

### mediapose 사용법

In [ ]:
mpPose = mp.solutions.pose
pose = mpPose.Pose()

img=cv2.imread("/content/drive/MyDrive/cv_data/P001/A001_P001_G001_C001/0.jpg")
imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img2 = cv2.resize(imgRGB, dsize=(480, 320), interpolation=cv2.INTER_AREA)
results = pose.process(img2)
result=results.pose_landmarks
pose_landmarks = [[lmk.x, lmk.y] for lmk in result.landmark]
data=np.array(pose_landmarks)
data2=pd.DataFrame(data,columns=["x","y"])
data2["x"]=data2["x"]*480
data2["y"]=data2["y"]*320

In [ ]:
data2

,x,y
0,235.851517,98.765259
1,234.678426,92.748394
2,234.226141,92.517776
3,233.816028,92.250404
4,234.351525,92.602701
5,233.380237,92.092543
6,232.434869,91.603794
7,227.636318,91.142740
8,225.624347,90.422497
9,231.929226,102.248192


### mediapose 후 관절 재배치 및 재조합

In [ ]:
def arrange_skeleton(df):
  final_input=[]
  hip=((df.iloc[24][0]+df.iloc[23][0])/2,(df.iloc[24][1]+df.iloc[23][1])/2)
  final_input.append(np.float32(hip))
  right_hip=df.iloc[24]
  final_input.append(np.float32(right_hip))
  right_knee=df.iloc[26]
  final_input.append(np.float32(right_knee))
  right_foot=df.iloc[28]
  final_input.append(np.float32(right_foot))
  left_hip=df.iloc[23]
  final_input.append(np.float32(left_hip))
  left_knee=df.iloc[25]
  final_input.append(np.float32(left_knee))
  left_foot=df.iloc[27]
  final_input.append(np.float32(left_foot))
  spine=((df.iloc[12][0]+df.iloc[11][0])/2,(df.iloc[12][1]+df.iloc[11][1])/2)
  final_input.append(np.float32(spine))
  thorax=(spine[0]*2-(df.iloc[0][0]+spine[0])/2,spine[1]*2-(df.iloc[0][1]+spine[1])/2)
  final_input.append(np.float32(thorax))
  head=df.iloc[0]
  final_input.append(np.float32(head))
  left_shoulder=df.iloc[11]
  final_input.append(np.float32(left_shoulder))
  left_elbow=df.iloc[13]
  final_input.append(np.float32(left_elbow))
  left_wrist=df.iloc[15]
  final_input.append(np.float32(left_wrist))
  right_shoulder=df.iloc[12]
  final_input.append(np.float32(right_shoulder))
  right_elbow=df.iloc[14]
  final_input.append(np.float32(right_elbow))
  right_wrist=df.iloc[16]
  final_input.append(np.float32(right_wrist))
  return final_input

#### 사용 test

In [ ]:
mpPose = mp.solutions.pose
pose = mpPose.Pose()

img=cv2.imread("/content/drive/MyDrive/cv_data/P001/A001_P001_G001_C001/0.jpg")
imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img2 = cv2.resize(imgRGB, dsize=(480, 320), interpolation=cv2.INTER_AREA)
results = pose.process(img2)
result=results.pose_landmarks
pose_landmarks = [[lmk.x, lmk.y] for lmk in result.landmark]
data=np.array(pose_landmarks)
data2=pd.DataFrame(data,columns=["x","y"])
data2["x"]=data2["x"]*480
data2["y"]=data2["y"]*320
ret=arrange_skeleton(data2)
a=pd.DataFrame(ret,columns=['x','y'])

In [ ]:
a=pd.DataFrame(ret,columns=['x','y'])

In [ ]:
a

,x,y
0,217.928238,178.237076
1,214.740021,179.834000
2,266.334656,188.599121
3,248.046692,247.206650
4,221.116455,176.640152
5,256.405762,193.280807
6,245.715775,243.224258
7,213.176010,111.190506
8,201.838257,117.403130
9,235.851517,98.765259


## 영상 데이터 스켈레톤 2D 변환

In [ ]:
mpPose = mp.solutions.pose
pose = mpPose.Pose()
def data_to_skeleton(path):
  img=cv2.imread(path,cv2.IMREAD_COLOR)
  imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img2 = cv2.resize(imgRGB, dsize=(480, 320), interpolation=cv2.INTER_AREA)
  results = pose.process(img2)
  result=results.pose_landmarks
  if result is not None:
    pose_landmarks = [[lmk.x, lmk.y] for lmk in result.landmark]
    data=np.array(pose_landmarks)
    data2=pd.DataFrame(data,columns=["x","y"])
    data2["x"]=data2["x"]*480
    data2["y"]=data2["y"]*320
    ret=arrange_skeleton(data2)
    a=pd.DataFrame(ret,columns=['x','y'])
    return a,True
  else:
    return None,False

### A053 행동 영상 추출

In [ ]:
main_path="/content/drive/MyDrive/cv_data/A053/"
for i in os.listdir(main_path):
  sub_path1=main_path+i+'/'
  writeflag=1
  if i in os.listdir("/content/drive/MyDrive/mppose/A0532/"):
      print("이미 처리한 파일입니다.")
      continue 
  temp=pd.DataFrame(columns=['x','y'])
  for k in range(0,30):
    filepath=sub_path1+str(k)+".jpg"
    print(filepath)
    x_temp,flag = data_to_skeleton(filepath)
    if(flag==False):
      writeflag=0
      break;
    else:
      temp=pd.concat([temp,x_temp],axis=0)
  if(writeflag==1):
    write_path="/content/drive/MyDrive/mppose/A0532/"+i
    try:
      os.mkdir(write_path)
      temp.to_csv(write_path+'/'+i+'.csv',index=False)
    except:
      print("이미 생성된 폴더입니다.\n")
  else:
    print("처리불가 : ",sub_path1)
    continue

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
/content/drive/MyDrive/cv_data/A053/A053_P091_G004_C001/4.jpg
/content/drive/MyDrive/cv_data/A053/A053_P091_G004_C001/5.jpg
/content/drive/MyDrive/cv_data/A053/A053_P091_G004_C001/6.jpg
/content/drive/MyDrive/cv_data/A053/A053_P091_G004_C001/7.jpg
/content/drive/MyDrive/cv_data/A053/A053_P091_G004_C001/8.jpg
/content/drive/MyDrive/cv_data/A053/A053_P091_G004_C001/9.jpg
/content/drive/MyDrive/cv_data/A053/A053_P091_G004_C001/10.jpg
/content/drive/MyDrive/cv_data/A053/A053_P091_G004_C001/11.jpg
/content/drive/MyDrive/cv_data/A053/A053_P091_G004_C001/12.jpg
/content/drive/MyDrive/cv_data/A053/A053_P091_G004_C001/13.jpg
/content/drive/MyDrive/cv_data/A053/A053_P091_G004_C001/14.jpg
/content/drive/MyDrive/cv_data/A053/A053_P091_G004_C001/15.jpg
/content/drive/MyDrive/cv_data/A053/A053_P091_G004_C001/16.jpg
/content/drive/MyDrive/cv_data/A053/A053_P091_G004_C001/17.jpg
/content/drive/MyDrive/cv_data/A053/A053_P091_G004_C001/18.jpg
/content/drive/MyDrive/cv

### P001 영상 추출

In [ ]:
main_path="/content/drive/MyDrive/cv_data/P001/"
for i in os.listdir(main_path):
  sub_path1=main_path+i+'/'
  writeflag=1
  if i in os.listdir("/content/drive/MyDrive/mppose/P001/"):
    print("이미 처리한 파일입니다.")
    continue
  if str(i[2:4]) in ["44","45","46","47","48","49","50","51"]:
    continue
  temp=pd.DataFrame(columns=['x','y'])
  for k in range(0,30):
    filepath=sub_path1+str(k)+".jpg"
    print(filepath)
    x_temp,flag = data_to_skeleton(filepath)
    if(flag==False):
      writeflag=0
      break;
    else:
      temp=pd.concat([temp,x_temp],axis=0)
  if(writeflag==1):
    write_path="/content/drive/MyDrive/mppose/P001/"+i
    try:
      os.mkdir(write_path)
      temp.to_csv(write_path+'/'+i+'.csv',index=False)
    except:
      print("이미 생성된 폴더입니다.\n")
  else:
    print("처리불가 : ",sub_path1)
    continue

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
/content/drive/MyDrive/cv_data/P001/A003_P001_G003_C005/23.jpg
/content/drive/MyDrive/cv_data/P001/A003_P001_G003_C005/24.jpg
/content/drive/MyDrive/cv_data/P001/A003_P001_G003_C005/25.jpg
/content/drive/MyDrive/cv_data/P001/A003_P001_G003_C005/26.jpg
/content/drive/MyDrive/cv_data/P001/A003_P001_G003_C005/27.jpg
/content/drive/MyDrive/cv_data/P001/A003_P001_G003_C005/28.jpg
/content/drive/MyDrive/cv_data/P001/A003_P001_G003_C005/29.jpg
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
/content/drive/MyDrive/cv_data/P001/A004_P001_G001_C001/0.jpg
처리불가 :  /content/drive/MyDrive/cv_data/P001/A004_P001_G001_C001/
이미 처리한 파일입니다.
이미 처리한 파일입니다.
이미 처리한 파일입니다.
/content/drive/MyDrive/cv_data/P001/A004_P001_G001_C005/0.jpg
/content/drive/MyDrive/cv_data/P001/A004_P001_G001_C005/1.jpg
/content/drive/MyDrive/cv_data/P001/A004_P001_G001_C005/2.jpg
/content/drive/MyDrive/cv_data/P001/A004_P001_G001_C005/3.jpg
/content/drive/MyDrive/cv_data/P001/A004_P001_G001_C005/4.jpg
/c

### P002 영상 추출

In [ ]:
main_path="/content/drive/MyDrive/cv_data/P002/"
for i in os.listdir(main_path):
  sub_path1=main_path+i+'/'
  writeflag=1
  if i in os.listdir("/content/drive/MyDrive/mppose/P002/"):
    print("이미 처리한 파일입니다.")
    continue
  if str(i[2:4]) in ["44","45","46","47","48","49","50","51"]:
    continue
  temp=pd.DataFrame(columns=['x','y'])
  for k in range(0,30):
    filepath=sub_path1+str(k)+".jpg"
    print(filepath)
    x_temp,flag = data_to_skeleton(filepath)
    if(flag==False):
      writeflag=0
      break;
    else:
      temp=pd.concat([temp,x_temp],axis=0)
  if(writeflag==1):
    write_path="/content/drive/MyDrive/mppose/P002/"+i
    try:
      os.mkdir(write_path)
      temp.to_csv(write_path+'/'+i+'.csv',index=False)
    except:
      print("이미 생성된 폴더입니다.\n")
  else:
    print("처리불가 : ",sub_path1)
    continue

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
/content/drive/MyDrive/cv_data/P002/A035_P002_G003_C006/8.jpg
/content/drive/MyDrive/cv_data/P002/A035_P002_G003_C006/9.jpg
/content/drive/MyDrive/cv_data/P002/A035_P002_G003_C006/10.jpg
/content/drive/MyDrive/cv_data/P002/A035_P002_G003_C006/11.jpg
/content/drive/MyDrive/cv_data/P002/A035_P002_G003_C006/12.jpg
/content/drive/MyDrive/cv_data/P002/A035_P002_G003_C006/13.jpg
/content/drive/MyDrive/cv_data/P002/A035_P002_G003_C006/14.jpg
/content/drive/MyDrive/cv_data/P002/A035_P002_G003_C006/15.jpg
/content/drive/MyDrive/cv_data/P002/A035_P002_G003_C006/16.jpg
/content/drive/MyDrive/cv_data/P002/A035_P002_G003_C006/17.jpg
/content/drive/MyDrive/cv_data/P002/A035_P002_G003_C006/18.jpg
/content/drive/MyDrive/cv_data/P002/A035_P002_G003_C006/19.jpg
/content/drive/MyDrive/cv_data/P002/A035_P002_G003_C006/20.jpg
/content/drive/MyDrive/cv_data/P002/A035_P002_G003_C006/21.jpg
/content/drive/MyDrive/cv_data/P002/A035_P002_G003_C006/22.jpg
/content/drive/MyDriv

### P003 영상 추출

In [ ]:
main_path="/content/drive/MyDrive/cv_data/P003/"
for i in os.listdir(main_path):
  sub_path1=main_path+i+'/'
  writeflag=1
  if i in os.listdir("/content/drive/MyDrive/mppose/P003/"):
    print("이미 처리한 파일입니다.")
    continue
  if str(i[2:4]) in ["44","45","46","47","48","49","50","51"]:
    continue
  temp=pd.DataFrame(columns=['x','y'])
  for k in range(0,30):
    filepath=sub_path1+str(k)+".jpg"
    print(filepath)
    x_temp,flag = data_to_skeleton(filepath)
    if(flag==False):
      writeflag=0
      break;
    else:
      temp=pd.concat([temp,x_temp],axis=0)
  if(writeflag==1):
    write_path="/content/drive/MyDrive/mppose/P003/"+i
    try:
      os.mkdir(write_path)
      temp.to_csv(write_path+'/'+i+'.csv',index=False)
    except:
      print("이미 생성된 폴더입니다.\n")
  else:
    print("처리불가 : ",sub_path1)
    continue

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
/content/drive/MyDrive/cv_data/P003/A043_P003_G002_C004/29.jpg
/content/drive/MyDrive/cv_data/P003/A043_P003_G002_C005/0.jpg
/content/drive/MyDrive/cv_data/P003/A043_P003_G002_C005/1.jpg
/content/drive/MyDrive/cv_data/P003/A043_P003_G002_C005/2.jpg
/content/drive/MyDrive/cv_data/P003/A043_P003_G002_C005/3.jpg
/content/drive/MyDrive/cv_data/P003/A043_P003_G002_C005/4.jpg
/content/drive/MyDrive/cv_data/P003/A043_P003_G002_C005/5.jpg
/content/drive/MyDrive/cv_data/P003/A043_P003_G002_C005/6.jpg
/content/drive/MyDrive/cv_data/P003/A043_P003_G002_C005/7.jpg
/content/drive/MyDrive/cv_data/P003/A043_P003_G002_C005/8.jpg
/content/drive/MyDrive/cv_data/P003/A043_P003_G002_C005/9.jpg
/content/drive/MyDrive/cv_data/P003/A043_P003_G002_C005/10.jpg
/content/drive/MyDrive/cv_data/P003/A043_P003_G002_C005/11.jpg
/content/drive/MyDrive/cv_data/P003/A043_P003_G002_C005/12.jpg
/content/drive/MyDrive/cv_data/P003/A043_P003_G002_C005/13.jpg
/content/drive/MyDrive/cv_dat

### P004 영상 추출

In [ ]:
main_path="/content/drive/MyDrive/cv_data/P004/"
for i in os.listdir(main_path):
  sub_path1=main_path+i+'/'
  writeflag=1
  if i in os.listdir("/content/drive/MyDrive/mppose/P004/"):
    print("이미 처리한 파일입니다.")
    continue
  if str(i[2:4]) in ["44","45","46","47","48","49","50","51"]:
    continue
  temp=pd.DataFrame(columns=['x','y'])
  for k in range(0,30):
    filepath=sub_path1+str(k)+".jpg"
    print(filepath)
    x_temp,flag = data_to_skeleton(filepath)
    if(flag==False):
      writeflag=0
      break;
    else:
      temp=pd.concat([temp,x_temp],axis=0)
  if(writeflag==1):
    write_path="/content/drive/MyDrive/mppose/P004/"+i
    try:
      os.mkdir(write_path)
      temp.to_csv(write_path+'/'+i+'.csv',index=False)
    except:
      print("이미 생성된 폴더입니다.\n")
  else:
    print("처리불가 : ",sub_path1)
    continue

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
/content/drive/MyDrive/cv_data/P004/A041_P004_G004_C005/17.jpg
/content/drive/MyDrive/cv_data/P004/A041_P004_G004_C005/18.jpg
/content/drive/MyDrive/cv_data/P004/A041_P004_G004_C005/19.jpg
/content/drive/MyDrive/cv_data/P004/A041_P004_G004_C005/20.jpg
/content/drive/MyDrive/cv_data/P004/A041_P004_G004_C005/21.jpg
/content/drive/MyDrive/cv_data/P004/A041_P004_G004_C005/22.jpg
/content/drive/MyDrive/cv_data/P004/A041_P004_G004_C005/23.jpg
/content/drive/MyDrive/cv_data/P004/A041_P004_G004_C005/24.jpg
/content/drive/MyDrive/cv_data/P004/A041_P004_G004_C005/25.jpg
/content/drive/MyDrive/cv_data/P004/A041_P004_G004_C005/26.jpg
/content/drive/MyDrive/cv_data/P004/A041_P004_G004_C005/27.jpg
/content/drive/MyDrive/cv_data/P004/A041_P004_G004_C005/28.jpg
/content/drive/MyDrive/cv_data/P004/A041_P004_G004_C005/29.jpg
/content/drive/MyDrive/cv_data/P004/A041_P004_G004_C006/0.jpg
/content/drive/MyDrive/cv_data/P004/A041_P004_G004_C006/1.jpg
/content/drive/MyDriv

### 데이터 패치

In [ ]:
def fetch_x_y_dataA0532(path):
  xa=np.zeros([1,16,2])
  y=[]

  for i in os.listdir(path+"A0532/"):
    x=pd.read_csv("./drive/MyDrive/mppose/A0532/"+i+'/'+i+'.csv')
    a=np.array(x[0:16]).reshape(1,16,2)
    for j in range(29):
      temp=np.array(x[16*(j+1):16*(j+2)]).reshape(1,16,2)
      a=np.concatenate([a,temp],axis=0)
    #a.shape=(30,16,2) 
    if (not np.isnan(a).any()):
      xa=np.concatenate([xa,a],axis=0)
      y.append(int(i[2:4]))
      
  return xa[1:],y 

In [ ]:
def fetch_x_y_data01(path):
  xa=np.zeros([1,16,2])
  y=[]

  for i in os.listdir(path+"P001/"):
    x=pd.read_csv("./drive/MyDrive/mppose/P001/"+i+'/'+i+'.csv')
    a=np.array(x[0:16]).reshape(1,16,2)
    for j in range(29):
      temp=np.array(x[16*(j+1):16*(j+2)]).reshape(1,16,2)
      a=np.concatenate([a,temp],axis=0)
    #a.shape=(30,16,2) 
    if (not np.isnan(a).any()):
      xa=np.concatenate([xa,a],axis=0)
      y.append(int(i[2:4]))
      
  return xa[1:],y 

In [ ]:
def fetch_x_y_data02(path):
  xa=np.zeros([1,16,2])
  y=[]

  for i in os.listdir(path+"P002/"):
    x=pd.read_csv("./drive/MyDrive/mppose/P002/"+i+'/'+i+'.csv')
    a=np.array(x[0:16]).reshape(1,16,2)
    for j in range(29):
      temp=np.array(x[16*(j+1):16*(j+2)]).reshape(1,16,2)
      a=np.concatenate([a,temp],axis=0)
    #a.shape=(30,16,2) 
    if (not np.isnan(a).any()):
      xa=np.concatenate([xa,a],axis=0)
      y.append(int(i[2:4]))
      
  return xa[1:],y 

In [ ]:
def fetch_x_y_data03(path):
  xa=np.zeros([1,16,2])
  y=[]

  for i in os.listdir(path+"P003/"):
    x=pd.read_csv("./drive/MyDrive/mppose/P003/"+i+'/'+i+'.csv')
    a=np.array(x[0:16]).reshape(1,16,2)
    for j in range(29):
      temp=np.array(x[16*(j+1):16*(j+2)]).reshape(1,16,2)
      a=np.concatenate([a,temp],axis=0)
    #a.shape=(30,16,2) 
    if (not np.isnan(a).any()):
      xa=np.concatenate([xa,a],axis=0)
      y.append(int(i[2:4]))
      
  return xa[1:],y 

In [ ]:
def fetch_x_y_data04(path):
  xa=np.zeros([1,16,2])
  y=[]

  for i in os.listdir(path+"P004/"):
    x=pd.read_csv("./drive/MyDrive/mppose/P004/"+i+'/'+i+'.csv')
    a=np.array(x[0:16]).reshape(1,16,2)
    for j in range(29):
      temp=np.array(x[16*(j+1):16*(j+2)]).reshape(1,16,2)
      a=np.concatenate([a,temp],axis=0)
    #a.shape=(30,16,2) 
    if (not np.isnan(a).any()):
      xa=np.concatenate([xa,a],axis=0)
      y.append(int(i[2:4]))
      
  return xa[1:],y 

In [ ]:
path="./drive/MyDrive/mppose/"
x1,y1=fetch_x_y_data01(path)
x2,y2=fetch_x_y_data02(path)
x3,y3=fetch_x_y_data03(path)
x4,y4=fetch_x_y_data04(path)
x5,y5=fetch_x_y_dataA0532(path)

In [ ]:
y12=np.array(y1)
y22=np.array(y2)
y32=np.array(y3)
y42=np.array(y4)
y52=np.array(y5)
y2d=np.concatenate([y12,y22,y32,y42,y52],axis=0)
x2d=np.concatenate([x1,x2,x3,x4,x5],axis=0)

In [ ]:
np.unique(y2d)

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 52, 53, 54, 55])

In [ ]:
y_re=y2d

라벨링 재조정

In [ ]:
for i,value in enumerate(y_re):
  if(value==52):
    y_re[i]=44
  if(value==53):
    y_re[i]=45
  if(value==54):
    y_re[i]=46
  if(value==55):
    y_re[i]=47
for i,value in enumerate(y_re):
  if(value==45):
    y_re[i]=1
  else:
    y_re[i]=0  

3d pose baseline 로드 및 적합

In [ ]:
my3d=keras.models.load_model("./drive/MyDrive/save_model/3dpose")

In [ ]:
xplus3=my3d(x2d)

In [ ]:
xplus3.shape

TensorShape([115800, 51])

In [ ]:
y_re.shape

(3860,)

In [ ]:
xplus3=np.array(xplus3)

3차원 골격 구조로 변환

In [ ]:
xplus2=xplus3.reshape(3860,30,17,3)

### STGCN

In [ ]:
class Graph:
    def __init__(self, labeling_mode='spatial'):
        self.num_node = 17
        self.self_link = [(i, i) for i in range(self.num_node)]
        self.inward_ori_index = [(1, 2), (1,5),(1,9),(2,3),(3,4),(5,6),(6,7),(8,9),(8,10),(8,12),(8,15),(10,11),(12,13),(13,14),(15,16),(16,17)]
        self.inward = [(i - 1, j - 1) for (i, j) in self.inward_ori_index]
        self.outward = [(j, i) for (i, j) in self.inward]
        self.neighbor = self.inward+self.outward
        self.A = self.get_adjacency_matrix(labeling_mode)

    def edge2mat(self,link, num_node):
        A = np.zeros((num_node, num_node))
        for i, j in link:
            A[j, i] = 1
        return A
    def normalize_digraph(self,A):  
        Dl = np.sum(A, 0)
        h, w = A.shape
        Dn = np.zeros((w, w))
        for i in range(w):
            if Dl[i] > 0:
                Dn[i, i] = Dl[i] ** (-1)
        AD = np.dot(A, Dn)
        return AD

    def get_spatial_graph(self,num_node, self_link, inward, outward):
        I = self.edge2mat(self_link, num_node)
        In = self.normalize_digraph(self.edge2mat(inward, num_node))
        Out = self.normalize_digraph(self.edge2mat(outward, num_node))
        A = np.stack((I, In, Out))
        return A

    def get_adjacency_matrix(self, labeling_mode=None):
        if labeling_mode is None:
            return self.A
        if labeling_mode == 'spatial':
            A = self.get_spatial_graph(self.num_node, self.self_link, self.inward, self.outward)
        else:
            raise ValueError()
        return A

In [ ]:
import tensorflow as tf
import numpy as np

REGULARIZER = tf.keras.regularizers.l2(l=0.0001)
INITIALIZER = tf.keras.initializers.VarianceScaling(scale=2.,
                                                    mode="fan_out",
                                                    distribution="truncated_normal")

In [ ]:
class SGCN(tf.keras.Model):
    def __init__(self, filters, kernel_size=3):
        super().__init__()
        self.kernel_size = kernel_size
        self.conv = tf.keras.layers.Conv2D(filters*kernel_size,
                                           kernel_size=3,
                                           padding='same',
                                           kernel_initializer=INITIALIZER,
                                           data_format='channels_first',
                                           kernel_regularizer=REGULARIZER)

    # N, C, T, V
    def call(self, x, A, training):
        x = self.conv(x)

        N = tf.shape(x)[0]
        C = tf.shape(x)[1]
        T = tf.shape(x)[2]
        V = tf.shape(x)[3]

        x = tf.reshape(x, [N, self.kernel_size, C//self.kernel_size, T, V])
        x = tf.einsum('nkctv,kvw->nctw', x, A)
        return x, A

In [ ]:
class STGCN(tf.keras.Model):
    #input x=(N,3,30,17) A = (3,17,17)
    def __init__(self, filters, kernel_size=[9, 3], stride=1, activation='relu',
                 residual=True, downsample=False):
        super().__init__()
        self.sgcn = SGCN(filters, kernel_size=kernel_size[1])

        self.tgcn = tf.keras.Sequential()
        self.tgcn.add(tf.keras.layers.BatchNormalization(axis=1))
        self.tgcn.add(tf.keras.layers.Activation(activation))
        self.tgcn.add(tf.keras.layers.Conv2D(filters,
                                                kernel_size=[kernel_size[0], 1],
                                                strides=[stride, 1],
                                                padding='same',
                                                kernel_initializer=INITIALIZER,
                                                data_format='channels_first',
                                                kernel_regularizer=REGULARIZER))
        self.tgcn.add(tf.keras.layers.BatchNormalization(axis=1))

        self.act = tf.keras.layers.Activation(activation)

        if not residual:
            self.residual = lambda x, training=False: 0
        elif residual and stride == 1 and not downsample:
            self.residual = lambda x, training=False: x
        else:
            self.residual = tf.keras.Sequential()
            self.residual.add(tf.keras.layers.Conv2D(filters,
                                                        kernel_size=[1, 1],
                                                        strides=[stride, 1],
                                                        padding='same',
                                                        kernel_initializer=INITIALIZER,
                                                        data_format='channels_first',
                                                        kernel_regularizer=REGULARIZER))
            self.residual.add(tf.keras.layers.BatchNormalization(axis=1))

    def call(self, x, A, training):
        res = self.residual(x, training=training)
        x, A = self.sgcn(x, A, training=training)
        x = self.tgcn(x, training=training)
        x += res
        x = self.act(x)
        return x, A
class mystgcn(tf.keras.Model,Graph):
    def __init__(self, num_classes=2):
        super().__init__()
        graph = Graph()
        self.A = tf.constant(graph.A,
                             dtype=tf.float32,
                             name='adjacency_matrix')

        self.data_bn = tf.keras.layers.BatchNormalization(axis=1)

        self.STGCN_layers = []
        self.STGCN_layers.append(STGCN(64, residual=False))
        self.STGCN_layers.append(STGCN(64))
        self.STGCN_layers.append(STGCN(64))
        self.STGCN_layers.append(STGCN(64))
        self.STGCN_layers.append(STGCN(128, stride=2, downsample=True))
        self.STGCN_layers.append(STGCN(128))
        self.STGCN_layers.append(STGCN(128))
        self.STGCN_layers.append(STGCN(256, stride=2, downsample=True))
        self.STGCN_layers.append(STGCN(256))
        self.STGCN_layers.append(STGCN(256))

        self.pool = tf.keras.layers.GlobalAveragePooling2D(data_format='channels_first')

        self.logits = tf.keras.layers.Conv1D(num_classes,
                                             kernel_size=1,
                                             padding='same',
                                             kernel_initializer=INITIALIZER,
                                             data_format='channels_first',
                                             kernel_regularizer=REGULARIZER)

    def call(self, x, training):
        x=x[:,:,:,:,tf.newaxis]
        N = tf.shape(x)[0]
        C = tf.shape(x)[1]
        T = tf.shape(x)[2]
        V = tf.shape(x)[3]
        M = 1

        x = tf.transpose(x, perm=[0, 4, 3, 1, 2])
        x = tf.reshape(x, [N * M, V * C, T])
        x = self.data_bn(x, training=training)
        x = tf.reshape(x, [N, M, V, C, T])
        x = tf.transpose(x, perm=[0, 1, 3, 4, 2])
        x = tf.reshape(x, [N * M, C, T, V]) #(N,3,30,17) A = (3,17,17)

        A = self.A
        for layer in self.STGCN_layers:
            x, A = layer(x, A, training=training)

        # N*M,C,T,V
        x = self.pool(x)
        x = x[:,:,tf.newaxis]
        x = self.logits(x)
        x = tf.reshape(x, [N, -1])

        return x

In [ ]:
xall=xplus2
yall=y_re

In [ ]:
stgcn=mystgcn()
y=tf.one_hot(yall,depth=2)
y_1=np.array(y)
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(xall,y_1,test_size=0.2, random_state=42,stratify=y_1)

x_train2=tf.stack(x_train)
x_train3=tf.transpose(x_train2,perm=[0,3,1,2])
x_test2=tf.stack(x_test)
x_test3=tf.transpose(x_test2,perm=[0,3,1,2])
y_train2=tf.stack(y_train)
y_test2=tf.stack(y_test)

early_cb=keras.callbacks.EarlyStopping(patience=20,restore_best_weights=True)
opt= tf.keras.optimizers.Adam()
myloss=tf.keras.losses.CategoricalCrossentropy(from_logits=True)
stgcn.compile(loss=myloss,optimizer=opt,metrics=['accuracy'])

In [ ]:
history=stgcn.fit(x_train3,y_train2,epochs=200,validation_data=(x_test3,y_test2),callbacks=[early_cb])

Epoch 1/200
97/97 [==============================] - 11s 72ms/step - loss: 2.3989 - accuracy: 0.8718 - val_loss: 2.1742 - val_accuracy: 0.6244
Epoch 2/200
97/97 [==============================] - 6s 64ms/step - loss: 0.8222 - accuracy: 0.9398 - val_loss: 1.1053 - val_accuracy: 0.8122
Epoch 3/200
97/97 [==============================] - 6s 63ms/step - loss: 0.8565 - accuracy: 0.9336 - val_loss: 0.8853 - val_accuracy: 0.8977
Epoch 4/200
97/97 [==============================] - 6s 62ms/step - loss: 0.7324 - accuracy: 0.9582 - val_loss: 0.9419 - val_accuracy: 0.8407
Epoch 5/200
97/97 [==============================] - 6s 64ms/step - loss: 0.6987 - accuracy: 0.9657 - val_loss: 0.8474 - val_accuracy: 0.9249
Epoch 6/200
97/97 [==============================] - 6s 64ms/step - loss: 0.6899 - accuracy: 0.9615 - val_loss: 0.6509 - val_accuracy: 0.9767
Epoch 7/200
97/97 [==============================] - 6s 63ms/step - loss: 0.6503 - accuracy: 0.9712 - val_loss: 0.8179 - val_accuracy: 0.9301
Epoch

실제 테스트 데이터 평가

In [ ]:
A = [
   [
      [-0.41413915, -0.29039192, -0.49717197, -0.87047786, -0.56873685, -0.5278813, -0.70815706, 0.049633607, -0.18491864, 0.15681939, 0.37295035, -0.37152416, -0.30378792, -0.3225313, 0.28056553, 0.29220566, -0.13667612], 
      [-0.41413915, -0.29039192, -0.49717197, -0.87047786, -0.56873685, -0.5278813, -0.70815706, 0.049633607, -0.18491864, 0.15681939, 0.37295035, -0.37152416, -0.30378792, -0.3225313, 0.28056553, 0.29220566, -0.13667612], 
      [-0.39808547, -0.28079796, -0.4050295, -0.7454031, -0.5516979, -0.49272513, -0.7147099, 0.08914657, -0.15558149, 0.20252092, 0.42409924, -0.3443033, -0.3414305, -0.39528015, 0.31053945, 0.33667606, -0.0018924922], 
      [-0.5609822, -0.43581092, -0.5043422, -0.76387215, -0.71530885, -0.6524985, -0.8722607, -0.08884952, -0.32798782, 0.022019997, 0.21967945, -0.5493245, -0.57460463, -0.59160966, 0.1632282, 0.25528508, 0.016932711], 
      [-0.5609822, -0.43581092, -0.5043422, -0.76387215, -0.71530885, -0.6524985, -0.8722607, -0.08884952, -0.32798782, 0.022019997, 0.21967945, -0.5493245, -0.57460463, -0.59160966, 0.1632282, 0.25528508, 0.016932711], 
      [-0.649832, -0.52111006, -0.56525457, -0.7694898, -0.80425066, -0.6974291, -0.90876484, -0.17429358, -0.41431192, -0.062603764, 0.12689431, -0.65016574, -0.68987894, -0.6926502, 0.09171783, 0.1989453, 0.032687694], 
      [-0.6228275, -0.49585235, -0.5524975, -0.77423424, -0.7766156, -0.69976443, -0.91139853, -0.13495825, -0.38108337, -0.02250143, 0.17447919, -0.60127, -0.6411823, -0.6458648, 0.1249084, 0.22962461, 0.04567472], 
      [-0.6228275, -0.49585235, -0.5524975, -0.77423424, -0.7766156, -0.69976443, -0.91139853, -0.13495825, -0.38108337, -0.02250143, 0.17447919, -0.60127, -0.6411823, -0.6458648, 0.1249084, 0.22962461, 0.04567472], 
      [-0.5228697, -0.39850044, -0.47726348, -0.741395, -0.6809783, -0.59538895, -0.8246312, -0.04177338, -0.28594518, 0.07137322, 0.27858758, -0.5027235, -0.5155303, -0.54285437, 0.19866282, 0.281608, 0.008302629], 
      [-0.5228697, -0.39850044, -0.47726348, -0.741395, -0.6809783, -0.59538895, -0.8246312, -0.04177338, -0.28594518, 0.07137322, 0.27858758, -0.5027235, -0.5155303, -0.54285437, 0.19866282, 0.281608, 0.008302629], 
      [-0.32903773, -0.21440323, -0.480136, -0.7438494, -0.47849643, -0.4564499, -0.73014104, 0.15568072, -0.08166045, 0.26752004, 0.4905029, -0.24399981, -0.24368155, -0.33385786, 0.3449038, 0.35678995, -0.006749481], 
      [-0.35576805, -0.23562099, -0.5116465, -0.83182836, -0.5058607, -0.4736118, -0.70111, 0.12675072, -0.11128072, 0.23676202, 0.46615502, -0.25780332, -0.19050792, -0.22951898, 0.33518362, 0.36577213, -0.04223159], 
      [-0.35576805, -0.23562099, -0.5116465, -0.83182836, -0.5058607, -0.4736118, -0.70111, 0.12675072, -0.11128072, 0.23676202, 0.46615502, -0.25780332, -0.19050792, -0.22951898, 0.33518362, 0.36577213, -0.04223159], 
      [-0.37036926, -0.24728943, -0.47085842, -0.8547857, -0.5333263, -0.49782208, -0.69752324, 0.11434626, -0.12923867, 0.22762431, 0.45646712, -0.3130318, -0.28261754, -0.34975007, 0.3399285, 0.33836746, -0.10946001], 
      [-0.38192406, -0.26955014, -0.3603597, -0.6563314, -0.53131604, -0.46594438, -0.6794448, 0.06648688, -0.16097462, 0.17347258, 0.36661375, -0.3639333, -0.36768013, -0.43271068, 0.27729815, 0.33506966, 0.040365115], 
      [-0.38192406, -0.26955014, -0.3603597, -0.6563314, -0.53131604, -0.46594438, -0.6794448, 0.06648688, -0.16097462, 0.17347258, 0.36661375, -0.3639333, -0.36768013, -0.43271068, 0.27729815, 0.33506966, 0.040365115], 
      [-0.6362649, -0.50999206, -0.53517985, -0.7590134, -0.78912604, -0.71167916, -0.95277494, -0.15751182, -0.39788792, -0.044600166, 0.14297102, -0.6271857, -0.6770131, -0.6893539, 0.102644384, 0.22599912, 0.049732067], 
      [-0.6717967, -0.54597896, -0.5730568, -0.7640367, -0.8208291, -0.72403276, -0.9448781, -0.19365214, -0.43381518, -0.081391074, 0.106038064, -0.65871704, -0.7118236, -0.70617044, 0.067945994, 0.18561737, 0.049255706], 
      [-0.6717967, -0.54597896, -0.5730568, -0.7640367, -0.8208291, -0.72403276, -0.9448781, -0.19365214, -0.43381518, -0.081391074, 0.106038064, -0.65871704, -0.7118236, -0.70617044, 0.067945994, 0.18561737, 0.049255706], 
      [-0.624211, -0.4983856, -0.5351541, -0.7802915, -0.7764144, -0.70904124, -0.9487705, -0.121636316, -0.3722335, -0.004919732, 0.19099894, -0.5882276, -0.6412144, -0.6548449, 0.13785648, 0.26684633, 0.07630723], 
      [-0.47422913, -0.34829563, -0.42862287, -0.723886, -0.63614285, -0.53610796, -0.75745046, -0.005836265, -0.2427946, 0.10699141, 0.3126436, -0.46648395, -0.49522752, -0.5576866, 0.22328633, 0.27840793, -0.016308412], 
      [-0.47422913, -0.34829563, -0.42862287, -0.723886, -0.63614285, -0.53610796, -0.75745046, -0.005836265, -0.2427946, 0.10699141, 0.3126436, -0.46648395, -0.49522752, -0.5576866, 0.22328633, 0.27840793, -0.016308412], 
      [-0.46079588, -0.3321241, -0.5079452, -0.83747035, -0.62202126, -0.569264, -0.78952914, 0.023177464, -0.21728607, 0.1373513, 0.35961086, -0.4213576, -0.44282502, -0.51489574, 0.24492002, 0.26488563, -0.11371265], 
      [-0.37742144, -0.25263494, -0.51185715, -0.82261926, -0.535527, -0.49755087, -0.7371524, 0.12111468, -0.12158481, 0.23779833, 0.4661614, -0.29699317, -0.30323407, -0.37566623, 0.3249069, 0.3369741, -0.05495435], 
      [-0.37742144, -0.25263494, -0.51185715, -0.82261926, -0.535527, -0.49755087, -0.7371524, 0.12111468, -0.12158481, 0.23779833, 0.4661614, -0.29699317, -0.30323407, -0.37566623, 0.3249069, 0.3369741, -0.05495435], 
      [-0.36744025, -0.24578021, -0.48096642, -0.81746584, -0.52514327, -0.50499684, -0.7107391, 0.10023992, -0.13270864, 0.21117207, 0.42782778, -0.32491854, -0.3181529, -0.38040835, 0.31334305, 0.3217281, -0.094425544], 
      [-0.65647066, -0.5266175, -0.5742421, -0.7633804, -0.818358, -0.7111486, -0.9034868, -0.2277785, -0.4486213, -0.118721806, 0.052913852, -0.70840365, -0.7952712, -0.7885569, 0.045585677, 0.14159507, -0.020900488], 
      [-0.66879183, -0.54305613, -0.58582604, -0.767446, -0.8204168, -0.7355896, -0.94218725, -0.22251023, -0.44915172, -0.11365754, 0.057929292, -0.69092053, -0.76923335, -0.7550818, 0.041862465, 0.16879545, 0.03304805], 
      [-0.66879183, -0.54305613, -0.58582604, -0.767446, -0.8204168, -0.7355896, -0.94218725, -0.22251023, -0.44915172, -0.11365754, 0.057929292, -0.69092053, -0.76923335, -0.7550818, 0.041862465, 0.16879545, 0.03304805], 
      [-0.6563447, -0.5277891, -0.56562936, -0.7666097, -0.8124543, -0.71650887, -0.9427912, -0.1929947, -0.42667875, -0.0800374, 0.103639856, -0.67085725, -0.75499964, -0.7579202, 0.068049565, 0.18022688, 0.029509917], 
   ],
   [
      [-0.4286962, -0.5707707, -0.80270296, -1.2464471, -0.32932118, -0.42232043, -1.0005176, 0.22455534, -0.060182076, 0.28625658, 0.41851795, 0.27497086, 0.11497157, -0.060271952, -0.09276353, 0.032695573, 0.4185391], 
      [-0.4286962, -0.5707707, -0.80270296, -1.2464471, -0.32932118, -0.42232043, -1.0005176, 0.22455534, -0.060182076, 0.28625658, 0.41851795, 0.27497086, 0.11497157, -0.060271952, -0.09276353, 0.032695573, 0.4185391], 
      [-0.44696498, -0.58972234, -0.877945, -1.309109, -0.350101, -0.46639115, -1.0342937, 0.19863877, -0.085556254, 0.2602463, 0.38287348, 0.24355814, 0.105864, -0.09331091, -0.08504234, -0.04341784, 0.31788647], 
      [-0.43435544, -0.56429833, -0.8657905, -1.3160677, -0.33949935, -0.45660955, -0.9925931, 0.1641827, -0.096321836, 0.21550554, 0.33330673, 0.18868981, 0.09707332, -0.07555945, -0.072934315, -0.103966676, 0.2427335], 
      [-0.43435544, -0.56429833, -0.8657905, -1.3160677, -0.33949935, -0.45660955, -0.9925931, 0.1641827, -0.096321836, 0.21550554, 0.33330673, 0.18868981, 0.09707332, -0.07555945, -0.072934315, -0.103966676, 0.2427335], 
      [-0.4869513, -0.61167485, -0.9018055, -1.3666058, -0.39618134, -0.48935318, -1.0228752, 0.0597101, -0.1789479, 0.10449, 0.21251178, 0.06332371, 1.02494145E-4, -0.14736746, -0.16008721, -0.21939822, 0.14119266], 
      [-0.45757067, -0.5859037, -0.88290626, -1.3359444, -0.36419177, -0.46570122, -1.0021901, 0.10019202, -0.14288439, 0.14584635, 0.2526225, 0.11729632, 0.044434503, -0.117676616, -0.12102215, -0.171764, 0.17758569], 
      [-0.45757067, -0.5859037, -0.88290626, -1.3359444, -0.36419177, -0.46570122, -1.0021901, 0.10019202, -0.14288439, 0.14584635, 0.2526225, 0.11729632, 0.044434503, -0.117676616, -0.12102215, -0.171764, 0.17758569], 
      [-0.45640954, -0.5933856, -0.883515, -1.3713077, -0.3575105, -0.5144154, -1.078666, 0.18946576, -0.092391044, 0.24862953, 0.3756074, 0.22019958, 0.121448584, -0.072036006, -0.065547764, -0.088876136, 0.25498724], 
      [-0.45640954, -0.5933856, -0.883515, -1.3713077, -0.3575105, -0.5144154, -1.078666, 0.18946576, -0.092391044, 0.24862953, 0.3756074, 0.22019958, 0.121448584, -0.072036006, -0.065547764, -0.088876136, 0.25498724], 
      [-0.49880844, -0.64104354, -1.040555, -1.5046126, -0.39439648, -0.71649706, -1.2791393, 0.29831138, -0.042137273, 0.38019603, 0.5055797, 0.37722635, 0.29986823, 0.15651506, 0.026526406, 0.07451258, 0.4477554], 
      [-0.48197305, -0.6277188, -0.9333964, -1.4054794, -0.37492687, -0.61966205, -1.193962, 0.27914727, -0.046638858, 0.3555972, 0.4898106, 0.36032698, 0.24836715, 0.10523567, -0.028429925, 0.10693581, 0.51757324], 
      [-0.48197305, -0.6277188, -0.9333964, -1.4054794, -0.37492687, -0.61966205, -1.193962, 0.27914727, -0.046638858, 0.3555972, 0.4898106, 0.36032698, 0.24836715, 0.10523567, -0.028429925, 0.10693581, 0.51757324], 
      [-0.4419797, -0.59111094, -0.88638866, -1.3476543, -0.34136558, -0.5105261, -1.0857983, 0.28064445, -0.031028036, 0.3519574, 0.489204, 0.34666282, 0.18662143, 0.0067577064, -0.038106337, 0.061717764, 0.466483], 
      [-0.41332155, -0.54406023, -0.83146936, -1.2833922, -0.3205997, -0.44986385, -1.0110781, 0.2228433, -0.056111407, 0.28221288, 0.39645186, 0.25521797, 0.15030853, -0.040424675, -0.025818318, -0.013270611, 0.31972545], 
      [-0.41332155, -0.54406023, -0.83146936, -1.2833922, -0.3205997, -0.44986385, -1.0110781, 0.2228433, -0.056111407, 0.28221288, 0.39645186, 0.25521797, 0.15030853, -0.040424675, -0.025818318, -0.013270611, 0.31972545], 
      [-0.42775017, -0.55250907, -0.8650825, -1.3351222, -0.33411542, -0.47151655, -1.0096911, 0.16245416, -0.09424628, 0.21248102, 0.32133877, 0.17531607, 0.11477714, -0.04572258, -0.051997617, -0.11398918, 0.22275248], 
      [-0.44778556, -0.56929636, -0.87195635, -1.3346956, -0.35674772, -0.45652163, -1.0036755, 0.09975173, -0.13872364, 0.14504083, 0.24450658, 0.104491346, 0.055483673, -0.08569841, -0.10651906, -0.15833275, 0.19300748], 
      [-0.44778556, -0.56929636, -0.87195635, -1.3346956, -0.35674772, -0.45652163, -1.0036755, 0.09975173, -0.13872364, 0.14504083, 0.24450658, 0.104491346, 0.055483673, -0.08569841, -0.10651906, -0.15833275, 0.19300748], 
      [-0.43947375, -0.56958055, -0.85872316, -1.3431294, -0.3413324, -0.48490936, -1.0106167, 0.15124676, -0.1065598, 0.20225736, 0.3132419, 0.17881364, 0.106862135, -0.0695608, -0.07083917, -0.1365859, 0.18990894], 
      [-0.4668501, -0.60392034, -0.8824413, -1.3877063, -0.3737082, -0.50893533, -1.0903324, 0.23033264, -0.07552458, 0.29741442, 0.43181145, 0.24598849, 0.13679232, -0.06727134, -0.03644736, -0.049940735, 0.3056363], 
      [-0.4668501, -0.60392034, -0.8824413, -1.3877063, -0.3737082, -0.50893533, -1.0903324, 0.23033264, -0.07552458, 0.29741442, 0.43181145, 0.24598849, 0.13679232, -0.06727134, -0.03644736, -0.049940735, 0.3056363], 
      [-0.48375392, -0.6305418, -0.9208748, -1.4323367, -0.38353723, -0.58242154, -1.1506002, 0.2634589, -0.061153416, 0.33894193, 0.4799877, 0.31289542, 0.18440934, -0.021930084, -0.03184177, -0.0051020943, 0.37579775], 
      [-0.47441846, -0.62316215, -0.9827328, -1.494005, -0.36796793, -0.688113, -1.2554805, 0.31695765, -0.022609353, 0.39830548, 0.54402715, 0.39771146, 0.32494187, 0.1457102, 0.0188092, 0.116117425, 0.5433332], 
      [-0.47441846, -0.62316215, -0.9827328, -1.494005, -0.36796793, -0.688113, -1.2554805, 0.31695765, -0.022609353, 0.39830548, 0.54402715, 0.39771146, 0.32494187, 0.1457102, 0.0188092, 0.116117425, 0.5433332], 
      [-0.4154985, -0.560208, -0.88767755, -1.3743458, -0.31510502, -0.5671127, -1.1248856, 0.31081626, -0.0014617629, 0.38531667, 0.5258608, 0.38402534, 0.27121872, 0.079476215, 0.01642026, 0.10058884, 0.4736228], 
      [-0.42097557, -0.5367527, -0.8908468, -1.3675928, -0.34102073, -0.4626156, -1.0171221, 0.1850684, -0.0757032, 0.23577914, 0.3482194, 0.1702715, 0.10020017, -0.03663136, -0.010571629, -0.09401948, 0.27421987], 
      [-0.41623, -0.53322446, -0.8704803, -1.3374238, -0.33055866, -0.47254398, -0.99277294, 0.14733575, -0.096016616, 0.1936664, 0.29668602, 0.14948793, 0.091892846, -0.04813794, -0.041969165, -0.12285728, 0.2237352], 
      [-0.41623, -0.53322446, -0.8704803, -1.3374238, -0.33055866, -0.47254398, -0.99277294, 0.14733575, -0.096016616, 0.1936664, 0.29668602, 0.14948793, 0.091892846, -0.04813794, -0.041969165, -0.12285728, 0.2237352], 
      [-0.44532064, -0.56696653, -0.8867509, -1.3764999, -0.35714686, -0.49084342, -1.0307889, 0.16224103, -0.101569995, 0.2158488, 0.32713008, 0.16303927, 0.10145836, -0.055338517, -0.038265586, -0.1336574, 0.21201214], 
   ],
   [
      [1.569141, 1.5204632, 1.6175196, 1.9074872, 1.5085108, 2.1456523, 2.099687, 1.7941139, 1.7135091, 1.79395, 1.9244843, 1.7806764, 1.2831113, 1.1741345, 1.7783539, 1.7685802, 1.924223], 
      [1.569141, 1.5204632, 1.6175196, 1.9074872, 1.5085108, 2.1456523, 2.099687, 1.7941139, 1.7135091, 1.79395, 1.9244843, 1.7806764, 1.2831113, 1.1741345, 1.7783539, 1.7685802, 1.924223], 
      [1.609215, 1.5599391, 1.6671462, 1.9596739, 1.5526862, 2.178107, 2.2295594, 1.7856724, 1.7238231, 1.7763693, 1.885606, 1.8013529, 1.3382838, 1.2020726, 1.8039411, 1.8219283, 1.9692817], 
      [1.6841013, 1.6399636, 1.7542944, 2.0340583, 1.6271555, 2.1670542, 2.2904766, 1.8387578, 1.7849, 1.8265792, 1.9304724, 1.8423297, 1.4028381, 1.2307837, 1.8851216, 1.8923826, 2.0392835], 
      [1.6841013, 1.6399636, 1.7542944, 2.0340583, 1.6271555, 2.1670542, 2.2904766, 1.8387578, 1.7849, 1.8265792, 1.9304724, 1.8423297, 1.4028381, 1.2307837, 1.8851216, 1.8923826, 2.0392835], 
      [1.6911001, 1.654875, 1.7765785, 2.0624328, 1.6300915, 2.133385, 2.2951221, 1.8386546, 1.7887115, 1.8223908, 1.9348896, 1.8294305, 1.3602796, 1.1635027, 1.91358, 1.9130408, 2.0656242], 
      [1.6958876, 1.6562796, 1.7635399, 2.0474086, 1.6364412, 2.1499624, 2.3080747, 1.8334928, 1.7871749, 1.8178298, 1.9220389, 1.8366479, 1.3838046, 1.192286, 1.9041883, 1.910398, 2.0612602], 
      [1.6958876, 1.6562796, 1.7635399, 2.0474086, 1.6364412, 2.1499624, 2.3080747, 1.8334928, 1.7871749, 1.8178298, 1.9220389, 1.8366479, 1.3838046, 1.192286, 1.9041883, 1.910398, 2.0612602], 
      [1.6575024, 1.6083213, 1.7591541, 2.0212224, 1.6016264, 2.1747246, 2.270484, 1.8027339, 1.7537997, 1.7888849, 1.8789127, 1.8108076, 1.3494062, 1.1728785, 1.8555539, 1.8865261, 2.0133362], 
      [1.6575024, 1.6083213, 1.7591541, 2.0212224, 1.6016264, 2.1747246, 2.270484, 1.8027339, 1.7537997, 1.7888849, 1.8789127, 1.8108076, 1.3494062, 1.1728785, 1.8555539, 1.8865261, 2.0133362], 
      [1.571744, 1.5292687, 1.7114923, 1.9789451, 1.4946634, 2.108829, 2.1195188, 1.7376537, 1.6905379, 1.7323527, 1.8654385, 1.6444974, 1.2532244, 1.2810602, 1.683258, 1.7180879, 1.8802466], 
      [1.5409455, 1.5046731, 1.6546879, 1.9375322, 1.4605584, 2.0945907, 2.033936, 1.7435522, 1.6815712, 1.7449462, 1.8884072, 1.6430831, 1.2116927, 1.2266542, 1.68862, 1.7104971, 1.8881938], 
      [1.5409455, 1.5046731, 1.6546879, 1.9375322, 1.4605584, 2.0945907, 2.033936, 1.7435522, 1.6815712, 1.7449462, 1.8884072, 1.6430831, 1.2116927, 1.2266542, 1.68862, 1.7104971, 1.8881938], 
      [1.5377628, 1.4905009, 1.604368, 1.8711814, 1.4694549, 2.1348805, 2.0662117, 1.7372024, 1.6687832, 1.7332698, 1.8601792, 1.7114017, 1.2461001, 1.1893215, 1.7087574, 1.7206194, 1.879562], 
      [1.6030331, 1.5605459, 1.6739067, 1.9439162, 1.5420642, 2.1253712, 2.1832964, 1.7928536, 1.7264485, 1.78631, 1.8926897, 1.7925233, 1.3511721, 1.2032382, 1.8192854, 1.8340816, 1.9552283], 
      [1.6030331, 1.5605459, 1.6739067, 1.9439162, 1.5420642, 2.1253712, 2.1832964, 1.7928536, 1.7264485, 1.78631, 1.8926897, 1.7925233, 1.3511721, 1.2032382, 1.8192854, 1.8340816, 1.9552283], 
      [1.676952, 1.6377981, 1.758012, 2.0362878, 1.6155772, 2.1203165, 2.2810025, 1.8273413, 1.7753391, 1.8131697, 1.9171298, 1.8234305, 1.3890557, 1.2027049, 1.8887424, 1.9102285, 2.0510905], 
      [1.6877995, 1.6558387, 1.7601335, 2.048238, 1.622441, 2.1079345, 2.287259, 1.8485059, 1.792399, 1.8340311, 1.9512596, 1.8389306, 1.3818759, 1.1873434, 1.9246137, 1.9361855, 2.0966938], 
      [1.6877995, 1.6558387, 1.7601335, 2.048238, 1.622441, 2.1079345, 2.287259, 1.8485059, 1.792399, 1.8340311, 1.9512596, 1.8389306, 1.3818759, 1.1873434, 1.9246137, 1.9361855, 2.0966938], 
      [1.6720356, 1.6339498, 1.7568727, 2.004163, 1.6092501, 2.119412, 2.2551398, 1.8176513, 1.7680448, 1.802911, 1.8975444, 1.8164277, 1.3839793, 1.1802831, 1.8887933, 1.9129148, 2.0436635], 
      [1.6035485, 1.5585537, 1.6842086, 1.9533081, 1.5402193, 2.1493824, 2.2229455, 1.7772996, 1.7152214, 1.7659755, 1.8635755, 1.7832459, 1.3092318, 1.1333535, 1.8190786, 1.8626907, 1.9961843], 
      [1.6035485, 1.5585537, 1.6842086, 1.9533081, 1.5402193, 2.1493824, 2.2229455, 1.7772996, 1.7152214, 1.7659755, 1.8635755, 1.7832459, 1.3092318, 1.1333535, 1.8190786, 1.8626907, 1.9961843], 
      [1.5742896, 1.525701, 1.6542804, 1.9116515, 1.5082858, 2.138091, 2.1417572, 1.7164474, 1.6709647, 1.7041757, 1.8088946, 1.7038189, 1.2500532, 1.1538279, 1.7227645, 1.7600797, 1.9008015], 
      [1.5339311, 1.4901165, 1.6672099, 1.9090567, 1.4563606, 2.102357, 2.0519466, 1.6809716, 1.6412094, 1.6740509, 1.794903, 1.6002811, 1.1910582, 1.1886319, 1.638993, 1.6670508, 1.8183987], 
      [1.5339311, 1.4901165, 1.6672099, 1.9090567, 1.4563606, 2.102357, 2.0519466, 1.6809716, 1.6412094, 1.6740509, 1.794903, 1.6002811, 1.1910582, 1.1886319, 1.638993, 1.6670508, 1.8183987], 
      [1.54125, 1.4906073, 1.6355078, 1.8937781, 1.4764094, 2.1049922, 2.0308394, 1.6970638, 1.6503718, 1.6895065, 1.805913, 1.6655083, 1.2335955, 1.1840326, 1.6724448, 1.6899304, 1.8285577], 
      [1.66096, 1.628798, 1.7169006, 1.997285, 1.5918391, 2.0880795, 2.237539, 1.819077, 1.7611321, 1.8030043, 1.9280322, 1.8082545, 1.3660693, 1.1869234, 1.8802887, 1.8952098, 2.0775106], 
      [1.6860046, 1.6547178, 1.754085, 2.0134377, 1.6199979, 2.0826867, 2.236434, 1.8298917, 1.7794557, 1.8139492, 1.9294667, 1.8210266, 1.3929332, 1.2045376, 1.8995941, 1.9054265, 2.063291], 
      [1.6860046, 1.6547178, 1.754085, 2.0134377, 1.6199979, 2.0826867, 2.236434, 1.8298917, 1.7794557, 1.8139492, 1.9294667, 1.8210266, 1.3929332, 1.2045376, 1.8995941, 1.9054265, 2.063291], 
      [1.6655427, 1.6327069, 1.7349092, 2.0042162, 1.5987961, 2.0914822, 2.24923, 1.8079805, 1.7581817, 1.7907308, 1.9018514, 1.79864, 1.3584096, 1.1629258, 1.8789263, 1.9044533, 2.0614247], 
   ],
]

In [ ]:
a=np.array(A)
a2=a[np.newaxis,:,:,:]

In [ ]:
a3=tf.stack(a2)
a4=tf.transpose(a3,perm=[0,3,1,2])

In [ ]:
stgcn.predict(a3)

array([[-2.9236894,  2.2346756]], dtype=float32)